In [45]:
import pandas as pd
from datetime import datetime, timedelta

# Función para convertir el timestamp a fecha legible
def convert_timestamp(timestamp):
    return datetime.utcfromtimestamp(timestamp / 1000).strftime('%Y-%m-%d')

# Cargar los datos desde los archivos JSON
ipc_europe = pd.read_json('../docs/investing ipc-mensual-europa-928.json', orient='records')
ipc_eeuu = pd.read_json('../docs/investing_ipc-mensual-eeu-69.json', orient='records')

# Crear DataFrame a partir de la columna 'data' y asignar nombres de columnas
ipc_eeuu_df = pd.DataFrame(ipc_eeuu['data'].to_list(), columns=['timestamp', 'change', 'trend'])
ipc_europe_df = pd.DataFrame(ipc_europe['data'].to_list(), columns=['timestamp', 'change', 'trend'])

# Convertir el timestamp a fecha legible para ambos DataFrames
ipc_eeuu_df['date'] = pd.to_datetime(ipc_eeuu_df['timestamp'] / 1000, unit='s').dt.strftime('%Y-%m-%dT%H:%M:%S')
ipc_europe_df['date'] = pd.to_datetime(ipc_europe_df['timestamp'] / 1000, unit='s').dt.strftime('%Y-%m-%dT%H:%M:%S')

# Ordenar por fecha
ipc_eeuu_df = ipc_eeuu_df.sort_values(by='date').reset_index(drop=True)
ipc_europe_df = ipc_europe_df.sort_values(by='date').reset_index(drop=True)

# Eliminar la columna 'timestamp'
ipc_eeuu_df.drop(columns=['timestamp'], inplace=True)
ipc_europe_df.drop(columns=['timestamp'], inplace=True)

# Función para rellenar fechas faltantes con interpolación
def fill_missing_dates(df):
    # Crear un rango completo de fechas desde la primera hasta la última fecha en el DataFrame
    full_date_range = pd.date_range(start=df['date'].min(), end=df['date'].max())
    
    # Crear un DataFrame con el rango completo de fechas
    full_df = pd.DataFrame({'date': full_date_range})
    
    # Convertir la columna 'date' al tipo datetime64[ns]
    df['date'] = pd.to_datetime(df['date'])
    
    # Unir el DataFrame original con el DataFrame de fechas completas
    merged_df = pd.merge(full_df, df, on='date', how='left')
    
    # Rellenar los valores de 'change' interpolando
    merged_df['change'] = merged_df['change'].interpolate()
    
    return merged_df


# Aplicar la función para rellenar fechas faltantes en ambos DataFrames
filled_ipc_eeuu_df = fill_missing_dates(ipc_eeuu_df)
filled_ipc_europe_df = fill_missing_dates(ipc_europe_df)

# Asignar valores a 'trend' basado en cambios en 'change'
def assign_trend_based_on_change(df):
    df['trend'] = (df['change'].diff() > 0).astype(int)
    df['trend'].mask(df['change'].diff() == 0, inplace=True)  # Si el cambio es 0, dejar 'trend' como NaN
    df['trend'].fillna(2, inplace=True)  # Reemplazar NaN con 2
    return df

filled_ipc_eeuu_df = assign_trend_based_on_change(filled_ipc_eeuu_df)
filled_ipc_europe_df = assign_trend_based_on_change(filled_ipc_europe_df)

# Guardar los DataFrames en archivos CSV con la fecha en formato ISO con 'T'
filled_ipc_eeuu_df['date'] = filled_ipc_eeuu_df['date'].dt.strftime('%Y-%m-%dT%H:%M:%S')
filled_ipc_europe_df['date'] = filled_ipc_europe_df['date'].dt.strftime('%Y-%m-%dT%H:%M:%S')

filled_ipc_eeuu_df.to_csv('../docs/filled_ipc_eeuu.csv', index=False)
filled_ipc_europe_df.to_csv('../docs/filled_ipc_europe.csv', index=False)
